# Business Problem

**Birliktelik Kuralları Nedir?**

Veri içerisindeki pattern'leri (ilişkileri, yapıları) bulmak için kullanılan kural tabanlı bir makine öğrenmesi tekniğidir.


Birliktelik analizi uygulamaları veri biliminde en çok karşımıza çıkan uygulamalardandır. Tavsiye sistemleri olarak da denk gelmiş olacaktır.


Bu uygulamalar karşınıza şu şekillerde gelmiş olabilir "o ürünü alan bu ürünü de aldı" ya da "o ilana bakanlar bu ilanlara da baktı" ya da "senin için çalma listesi oluşturduk" ya da "sıradaki video için önerilen video" gibi.

Bu senaryolar e-ticaret veri bilimi veri madenciliği çalışmaları kapsamında en sık karşımıza çıkacak olan senaryolar.

Türkiye'deki ve dünyadaki büyük e-ticaret şirketleri, spotify, amazon, netflix gibi biraz daha yakından bilebileceğimiz birçok platform tavsiye sistemlerini kullanmaktadır.

Peki özetle ne yapmaktadır bu birliktelik analizleri?


**Apriori Algoritması**

Bu alanda en çok kullanılan yöntemdir. 

Birliktelik kuralı analizi bazı metrikler incelenerek gerçekleştirilir:

- Destek (Support)

Support(X, Y) = Freq(X,Y)/N

X: ürün
Y: ürün
N: toplam alışveriş

- Güven (Confidence)

Confidence(X, Y) = Freq(X,Y) / Freq(X)

- Lift

Lift = Support (X, Y) / ( Support(X) * Support(Y) ) 



# Data Understanding

In [3]:
#!pip install mlxtend

In [4]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules

In [5]:
df = pd.read_csv('GroceryStoreDataSet.csv', sep=',',header=None)
data = list(df[0].apply(lambda x:x.split(',')))

In [6]:
items=[]
for row in data:
    uncommons = list(set(row) -set(items))
    for uc in uncommons:
        items.append(uc)
items
# yada 
#items = (df['0'].unique())
# items

['BISCUIT',
 'MILK',
 'BREAD',
 'CORNFLAKES',
 'TEA',
 'BOURNVITA',
 'MAGGI',
 'JAM',
 'COFFEE',
 'COCK',
 'SUGER']

In [7]:
df=pd.DataFrame(data)

In [8]:
encoded_vals = []
for index, row in df.iterrows(): 
    labels = {}
    uncommons = list(set(items) - set(row))
    commons = list(set(items).intersection(row))
    for uc in uncommons:
        labels[uc] = 0
    for com in commons:
        labels[com] = 1
    encoded_vals.append(labels)
type(encoded_vals)

list

In [9]:
df=pd.DataFrame(encoded_vals)

In [10]:
#!pip install mlxtend
#from mlxtend.preprocessing import TransactionEncoder

#te = TransactionEncoder()
#te_data = te.fit(data).transform(data)

x=pd.DataFrame(te_data)

x.head()

In [11]:
freq_items = apriori(df, min_support = 0.10, use_colnames = True, verbose = 1)

Processing 64 combinations | Sampling itemset size 4


In [54]:
freq_items.sort_values("support", ascending = False).head(20)

,support,itemsets
10,0.65,(BREAD)
2,0.40,(COFFEE)
6,0.35,(TEA)
8,0.35,(BISCUIT)
5,0.30,(SUGER)
1,0.30,(CORNFLAKES)
9,0.25,(MILK)
3,0.25,(MAGGI)
27,0.20,"(SUGER, BREAD)"
22,0.20,"(TEA, MAGGI)"


### Aksiyon Önerisi :
 Aksiyon Önerisi1 :Magazanin reyon ve raf duzenini müşterilerin en çok satın aldigi ürünleri daha kolay bir şekilde bulmalarıni
 saglayacak sekilde dizayn etmek.(support degeri en yuksek olanlar ekmek gibi)


In [55]:
ar=association_rules(freq_items, metric = "confidence", min_threshold = 0.6)
ar.sort_values("support", ascending = True).head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(COCK),(CORNFLAKES),0.15,0.30,0.1,0.666667,2.222222,0.055,2.1
25,"(COFFEE, BREAD)",(SUGER),0.15,0.30,0.1,0.666667,2.222222,0.055,2.1
26,"(MAGGI, JAM)",(BREAD),0.10,0.65,0.1,1.000000,1.538462,0.035,inf
27,"(MAGGI, BREAD)",(JAM),0.15,0.10,0.1,0.666667,6.666667,0.085,2.7
28,"(JAM, BREAD)",(MAGGI),0.10,0.25,0.1,1.000000,4.000000,0.075,inf


### Yorumlar
#yorum 1 Biskuvit ve kahve alimlarin butun alimlarin yuzde 10 ununda birlikte gorulmektedir.(antecedent support 0.1)
#yorum 2 Biskuvit ve kahve alimlarin yuzde 100 u conflakes de almistir(condidence 1)
#yorum 3 Jam içeren alışveriş işlemlerinde Jam içerdiginden dolayi Maggi ürünlerinin satışları 4 kat artiyor.(lift 4.0)

### Aksiyon Önerisi :

Recel alan müşterilerin ekmek alma olasılığı oldukça yüksektir(%100). 
Bu demek oluyor ki; Recel alan müşteriler genelde ekmek de almaktadır: Burada 2 turlu aksiyon onerilir. 
1) Recel aldıktan sonra ekmek reyonunun yolunu tutacaktır. 
Recel ve ekmek reyonları market içerisinde birbirinden uzak lokasyonlara konumlandırılarak, 
Recel alan müşterinin ekmek reyonuna giderken, pek çok reyon geçmesi zorunlu kılınabilir.
Böylece müşterinin yolculuğu sırasında herhangi bir ürün satın alma olasılığı arttırılabilir.
2) Yada tam tersi recel ve ekmek reyonlari yanyana konumlandirilir, musteri zten var olan egilim sayesinde satis artirma amaclanir
Bu iki case de belli birsure denenip sonuclara gore analiz yapip reyon degisikligine kara verilmesi mantiklidir.

In [11]:
ar[(ar.support>0.1)  &  (ar.confidence > 0.8) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,(COCK),(COFFEE),0.15,0.4,0.15,1.0,2.5,0.09,inf


In [18]:
freq_items.head(20)

,support,itemsets
0,0.35,(TEA)
1,0.40,(COFFEE)
2,0.15,(COCK)
3,0.30,(CORNFLAKES)
4,0.25,(MAGGI)
5,0.10,(JAM)
6,0.30,(SUGER)
7,0.20,(BOURNVITA)
8,0.65,(BREAD)
9,0.25,(MILK)


In [19]:
freq_items['length'] = freq_items['itemsets'].apply(lambda x:len(x))

In [21]:
freq_items.head(20)

,support,itemsets,length
0,0.35,(TEA),1
1,0.40,(COFFEE),1
2,0.15,(COCK),1
3,0.30,(CORNFLAKES),1
4,0.25,(MAGGI),1
5,0.10,(JAM),1
6,0.30,(SUGER),1
7,0.20,(BOURNVITA),1
8,0.65,(BREAD),1
9,0.25,(MILK),1


In [22]:
freq_items[(freq_items.support>0.1)  &  (freq_items.length == 2) ]

,support,itemsets,length
12,0.20,"(TEA, MAGGI)",2
14,0.20,"(TEA, BREAD)",2
16,0.15,"(COCK, COFFEE)",2
17,0.20,"(COFFEE, CORNFLAKES)",2
18,0.20,"(COFFEE, SUGER)",2
19,0.15,"(COFFEE, BREAD)",2
24,0.15,"(BISCUIT, CORNFLAKES)",2
26,0.15,"(BREAD, MAGGI)",2
30,0.20,"(SUGER, BREAD)",2
31,0.15,"(BREAD, BOURNVITA)",2
